In [7]:
import os
from glob import glob
from tqdm import tqdm
import pandas as pd
from PIL import Image
import torch
from torch import nn, optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from torchvision.datasets import ImageFolder
from efficientnet_pytorch import EfficientNet

In [14]:
# transform configuration
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.5, 0.5, 0.5], [0.2, 0.2, 0.2])
])

In [45]:
class VanillaEfficientNet(nn.Module):
    def __init__(self, n_class: int, freeze: bool = False):
        super(VanillaEfficientNet, self).__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b3')
        if freeze:
            self._freeze()
        self.batchnorm = nn.BatchNorm1d(num_features=1000)
        self.dropout = nn.Dropout()
        self.relu = nn.ReLU()
        self.linear = nn.Linear(in_features=1000, out_features=n_class)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.batchnorm(x)
        x = self.dropout(x)
        x = self.relu(x)
        output = self.linear(x)
        return output

    def _freeze(self):
        for param in self.efficientnet.parameters():
            param.requires_grad = False

In [47]:
model = VanillaEfficientNet(18)

Loaded pretrained weights for efficientnet-b3


In [46]:
temp = torch.rand(4,3,224,224)
temp.size()

torch.Size([4, 3, 224, 224])

In [48]:
model(temp)

tensor([[ 0.3997, -0.0692, -0.2884,  0.0105,  0.6515, -0.4172,  0.0310, -0.0247,
         -0.9070, -1.1667,  0.5936, -0.9430,  0.3337,  0.1926, -1.6241, -0.8383,
          0.5653, -0.2920],
        [-0.0125,  0.6508, -0.3419,  0.1676, -0.8270, -0.5039, -0.4700,  0.8726,
          0.0144,  0.1895, -0.1588, -0.0430, -0.6044, -0.7214, -0.2530,  0.2604,
          0.2683,  0.1307],
        [ 0.5126,  0.0911, -0.4083, -0.1728, -0.0151, -0.8613, -0.5094,  0.4684,
          0.7996, -0.5260,  0.1693,  0.0774,  1.3813,  0.8344, -0.2808, -0.0481,
         -0.0601,  0.2119],
        [ 0.5763,  0.8501, -0.4600,  0.7922,  0.3319, -0.5993,  0.1242, -0.9275,
         -0.3850,  0.1454,  0.9176, -0.4695, -0.3809,  1.0714, -0.4863, -1.0249,
         -0.5567,  0.3002]], grad_fn=<AddmmBackward>)

In [44]:
m = nn.BatchNorm1d(100)
m = nn.BatchNorm1d(100, affine=False)
input = torch.randn(2, 100)
output = m(input)

In [51]:
# simple pretrained model
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(512, 18)

# freeze
for name, param in model.named_parameters():
    if name not in ['fc.weight', 'fc.bias']:
        param.requires_grad = False # freeze

In [53]:
# train configuration
EPOCHS = 3
BATCH_SIZE = 64
LR = 0.0005

In [54]:
# compile
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)

In [59]:
# train phase
model.cuda()
model.train()

for epoch in range(EPOCHS):
    for imgs, labels in tqdm(loader, desc='Train'):
        imgs = imgs.cuda()
        labels = labels.cuda()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

Train: 100%|██████████| 885/885 [01:43<00:00,  8.54it/s]


In [75]:
# validation phase
model.eval()

total_corrects = 0
total_samples = 0
with torch.no_grad():
    for imgs, labels in tqdm(validloader, desc='Validation'):
        imgs = imgs.cuda()
        
        outputs = model(imgs)
        _, pred = torch.max(outputs, 1)
        pred = pred.cpu().numpy()
        labels = labels.cpu().numpy()
        
        total_corrects += (labels == pred).sum()
        total_samples += len(pred)

Validation: 100%|██████████| 178/178 [00:21<00:00,  8.38it/s]


In [76]:
print(f'Accuracy: {total_corrects/total_samples:.4f}')

Accuracy: 0.7873


In [103]:
class EvalDataset(Dataset):
    def __init__(self, transform=None):
        img_id_list = pd.read_csv('../input/data/eval/info.csv')['ImageID'] # to keep id order corresponding to that of submission
        self.img_paths = list(map(lambda x: os.path.join('../input/data/eval/images', x), img_id_list))
        self.transform = transform

    def __getitem__(self, index):
        name = os.path.basename(self.img_paths[index]) # ImageID
        image = Image.open(self.img_paths[index]) # image array
        if self.transform:
            image = self.transform(image) # transform(optional)
        return name, image

    def __len__(self):
        return len(self.img_paths)

In [113]:
# inference phase

evalset = EvalDataset(transform)
evalloader = DataLoader(evalset, batch_size=64, shuffle=False, drop_last=False)

model.eval()

name_list = []
pred_list = []

with torch.no_grad():
    for names, imgs in tqdm(evalloader):
        imgs = imgs.cuda()
        outputs = model(imgs)
        _, preds = torch.max(outputs, 1)
        
        pred_list.extend(preds.cpu().numpy().tolist())
        name_list.extend(names)

100%|██████████| 197/197 [01:26<00:00,  2.27it/s]


In [122]:
# check inference size
if len(name_list) == len(pred_list):
    print('Correct Inference Size:', len(name_list), len(pred_list))

Correct Inference Size: 12600 12600


In [125]:
# check inference order
submission_template = pd.read_csv('../input/data/eval/submission.csv')
id_order = submission_template['ImageID'].tolist()

allright = True
for i, j in zip(id_order, name_list):
    if i != j:
        allright = False
        print('Incorrect Order', i, j)

if allright:
    print('Correct ImageID Order')

Correct ImageID Order


In [117]:
# compose submission
submission = pd.DataFrame(dict(ImageID=name_list, ans=pred_list))
submission.to_csv('submission_please.csv', index=False)